<h2>
# Movies Classification Project
</center></h4> <h4>
 <img src="http://usblogs.pwc.com/emerging-technology/wp-content/uploads/2017/07/deep-learning-900x280.png" >


# Machine Learning Image

</h4>

Developed by [Sergio Castelblanco](http://www.linkedin.com/in/sergio-castelblanco/) - [Jesus Solano](http://www.jesussolano.com)

Andes University

version 2.1, July 9 2018

In [30]:
!pip install tqdm
!pip install livelossplot

In [0]:
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from skimage import io
from tqdm import tqdm
import os
%matplotlib inline

# Read Data

In [0]:
# Creates dataframes for images indices.

dataTraining = pd.read_csv("dataTraining.csv",index_col=0)
dataTesting = pd.read_csv("dataTesting.csv",index_col=0)

In [45]:
dataTraining.head()

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4
2582,1990,Narrow Margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6


In [46]:
dataTesting.head()

,year,title,plot
1,1999,Message in a Bottle,"who meets by fate , shall be sealed by fate ...."
4,1978,Midnight Express,"the true story of billy hayes , an american c..."
5,1996,Primal Fear,martin vail left the chicago da ' s office to ...
6,1950,Crisis,husband and wife americans dr . eugene and mr...
7,1959,The Tingler,the coroner and scientist dr . warren chapin ...


# Load Images

In [47]:
# Load all grey images.

print('\nLoading images for traning...')

trainingImagesGrey = []
for i in tqdm(dataTraining.index):
    trainingImagesGrey.append(io.imread(os.path.join('images_resize_gray', str(i) + '_resize_gray.jpeg')).flatten())
    
trainingImagesGrey = np.stack(trainingImagesGrey)

print('\nLoading images for testing...')

testingImagesGrey = []
for i in tqdm(dataTesting.index):
    testingImagesGrey.append(io.imread(os.path.join('images_resize_gray', str(i) + '_resize_gray.jpeg')).flatten())

testingImagesGrey = np.stack(images_testing)

  2%|▏         | 169/7895 [00:00<00:04, 1682.93it/s]


Loading images for traning...


  5%|▍         | 161/3383 [00:00<00:02, 1604.74it/s]


Loading images for testing...


100%|██████████| 3383/3383 [00:02<00:00, 1557.74it/s]


In [0]:
# Apply PCA to soft analysis.

pca = PCA(n_components=32)

trainingImagesGrey_pca = pca.fit_transform(trainingImagesGrey)
testingImagesGrey_pca = pca.transform(testingImagesGrey)

In [0]:
# Get labels for each image. 

dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))
le = MultiLabelBinarizer()
yGenres = le.fit_transform(dataTraining['genres'])

In [0]:
# Creates datasets for training and testing.   
xTrainImaGrey, xValidationImaGrey, yTrainImaGrey, yValidationImaGrey = train_test_split(trainingImagesGrey_pca, yGenres, test_size=0.33, random_state=22)

# Machine Learning Models

### Random Forest Classifier

In [51]:
# Creates a random forest classifier. 

mcRandomIma = OneVsRestClassifier(RandomForestClassifier(n_jobs=-1, n_estimators=100, max_depth=20, random_state=22,verbose=0))

# Train the model. 

mcRandomIma.fit(xTrainImaGrey,  yTrainImaGrey)

OneVsRestClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=22, verbose=0, warm_start=False),
          n_jobs=1)

In [52]:
# Score the AUC over the test set. 

y_pred_genres = mcRandomIma.predict_proba(xValidationImaGrey)

mcRandomIma_AUC=roc_auc_score(yValidationImaGrey, y_pred_genres, average='macro')

print('Random Forest model score is: ' , mcRandomIma_AUC , 'AUC.')

Random Forest model score is:  0.5644510037930798 AUC.


### Gradient Boost Classifier

In [54]:
from sklearn.ensemble import GradientBoostingClassifier

# Creates a random forest classifier. 

mcGradientIma = OneVsRestClassifier(GradientBoostingClassifier(n_estimators=100, max_depth=20, random_state=22,verbose=0))

# Train the model. 

mcGradientIma.fit(xTrainImaGrey,  yTrainImaGrey)

OneVsRestClassifier(estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=20,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=22, subsample=1.0, verbose=0,
              warm_start=False),
          n_jobs=1)

In [55]:
# Score the AUC over the test set. 

y_pred_genres = mcGradientIma.predict_proba(xValidationImaGrey)

mcGradientIma_AUC=roc_auc_score(yValidationImaGrey, y_pred_genres, average='macro')

print('Random Forest model score is: ' , mcGradientIma_AUC , 'AUC.')

Random Forest model score is:  0.5615014890922809 AUC.


# Apply models to test

In [0]:
# Print prediction for test (kaggle) data. 

yPredTest = mcRandomIma.predict_proba(testingImagesGrey_pca)

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

pd.DataFrame(yPredTest, index=dataTesting.index, columns=cols).to_csv('pred_genres_RF_img.csv', index_label='ID')

print('Results were printed.')